In [1]:
import configparser
import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofweek, hour, weekofyear, dayofmonth

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1583201800445_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://meg-sparkify-data/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### process_song_data

In [4]:
# get filepath to song data file
song_data = "song_data/*/*/*/*.json"
song_dataDF = spark.read.json(input_data + song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# extract columns to create songs table
songs_tableDF = song_dataDF['song_id', 'title', 'artist_id', 'year', 'duration']
songs_tableDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

In [6]:
# extract columns to create artists table and drop duplicates
artists_tableDF = song_dataDF['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
    artists_tableDF = artists_tableDF.drop_duplicates(subset=['artist_id'])

artists_tableDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)

In [8]:
artists_tableDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)

In [9]:
print(output_data + "song_data/artists.parquet")
                   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://meg-sparkify-data/song_data/artists.parquet'

In [ ]:
artists_tableDF.write.parquet(path=output_data + "song_data/artists.parquet", 
                                  mode="overwrite")


### process_log_data

In [21]:
# get filepath to log data file
log_data = "log_data/*/*/*.json"
# get filepath to song data file
song_data = "song_data/*/*/*/*.json"

log_dataDF = spark.read.json(input_data + log_data)

song_dataDF = spark.read.json(input_data + song_data)

song_dataDF.printSchema()

# filter by actions for song plays
log_dataDF = log_dataDF.where(col("page")=="NextSong")
log_dataDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = 

In [22]:
# extract columns for users table    
users_tableDF = log_dataDF['userId', 'firstName', 'lastName', 'gender', 'level', 'ts']
users_tableDF = users_tableDF.dropDuplicates(subset=["userId"])
print("users_tableDF")
users_tableDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

users_tableDF
root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)
 |-- ts: long (nullable = true)

In [12]:
print(output_data + "user_data/users.parquet")
# write artists table to parquet files
users_tableDF.write.parquet(path=output_data + "user_data/users.parquet", 
                                  mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://meg-sparkify-data/user_data/users.parquet

In [23]:
# ------ UDF  function
# extract datetime based on the timestamp from log dataframe
get_datetime_udf = udf(lambda ts: datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S'))

# add start_time, year, month columns to log data frame (needed by songsplay and time dateframe tables)
log_dataDF = log_dataDF.withColumn("start_time", get_datetime_udf(log_dataDF.ts)) 
print("log_dataDF")
log_dataDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

log_dataDF
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: string (nullable = true)

In [24]:
# extract columns to create time table
time_tableDF  = log_dataDF['start_time', dayofmonth('start_time').alias('day'), weekofyear('start_time').alias('week'), dayofweek('start_time').alias('weekday'), hour('start_time').alias('hour'), month('start_time').alias('month'), year('start_time').alias('year')]
time_tableDF = time_tableDF.drop_duplicates(subset=['start_time'])
print("time_tableDF")
time_tableDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time_tableDF
root
 |-- start_time: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)

In [30]:
print(output_data + "time_data/time.parquet")
# write time details back to parquet file partitioned by year and month
time_tableDF.write.parquet(partitionBy=["year","month"], 
                               path=output_data + "time_data/time.parquet",
                               mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://meg-sparkify-data/time_data/time.parquet

In [16]:
from pyspark.sql.functions import monotonically_increasing_id


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# extract columns from joined song and log datasets to create songplays table 
log_dataDF_J = log_dataDF.join(song_dataDF, (song_dataDF.title == log_dataDF.song) & (song_dataDF.artist_name == log_dataDF.artist))
print("log_dataDF_J")
log_dataDF_J.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

log_dataDF_J
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (

In [28]:
log_dataDF_J = log_dataDF_J.withColumn('songplay_id', monotonically_increasing_id())
songplays_tableDF = log_dataDF_J['songplay_id','start_time', 'userId', 'level', 'song_id', 'artist_id', 'sessionId', 'location', 'userAgent', month('start_time').alias('month'), year('start_time').alias('year')]


print("songplays_tableDF")
songplays_tableDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

songplays_tableDF
root
 |-- songplay_id: long (nullable = false)
 |-- start_time: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- location: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)

In [29]:
print(output_data + "songsplay_data/songsplay.parquet")
# write songs  to parquet file partitioned by year and month
songplays_tableDF.write.parquet(partitionBy=["year","month"], 
                                    path=output_data + "songsplay_data/songsplay.parquet", 
                                    mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://meg-sparkify-data/songsplay_data/songsplay.parquet